## Import necessary modules

In [ ]:
import sys 
import swat
import matplotlib
import matplotlib.pyplot as plt 
from matplotlib import dates as mpldates
import argparse
import os
import inspect
import re
import inspect
import tempfile
import collections
import numpy as np

## Set global variables
You may need to change these based on the CAS configuration you are using.

In [ ]:
# Set the CAS port  
DEFAULT_PORT = 5570  
# Host running CAS 
DEFAULT_HOST = "localhost" 
#DEFAULT_HOST = "sasserver.demo.sas.com" 
# Name of caslib to use 
DEFAULT_CASLIB = "mycas" 
# Set the protocol (None == autodetect) 
PROTOCOL = None 
# Directory to store temporary script in
TEMPDIR = "/tmp"
# Criterion for selecting the model
MODEL_SELECTION_CRITERION = "MAPE"
# Set path to data
DATA_PATH = "/opt/sas/viya/config/data/cas/default/public/"
# Path containing include scripts
#CODE_PATH = "/opt/sasinside/DemoData/R"
CODE_PATH = "/home/sasdemo"

## Add code to call the Timedata.RunTimeCode action

This includes creating the connection and loading the action set. 

In [ ]:
if __name__ == "__main__": 
    # Parse arguments 
    port, protocol, host, caslib = DEFAULT_PORT, PROTOCOL, DEFAULT_HOST, DEFAULT_CASLIB 

    # Create connection 
    conn = swat.CAS(host, port, caslib=caslib, 
                    protocol=protocol) 


    # Load needed action sets 
    conn.loadactionset(actionset="timedata") 


## Load the dataset into a CASLIB

In [ ]:
    # Specify path to data 
    indatadir = DATA_PATH 
    indata = "skinproduct" 
    if not conn.table.tableExists(table=indata).exists: 
        path_no_ext = os.path.join(indatadir, indata)
        if os.path.exists(path_no_ext + ".sashdat"):
            tbl = conn.upload_file(path_no_ext+".sashdat", casout={"name":indata})
        elif os.path.exists(path_no_ext + ".sas7bdat"):
            tbl = conn.upload_file(path_no_ext+".sas7bdat", casout={"name":indata})
        else: 
            tbl = conn.upload_file(path_no_ext+".csv",  
                                               casout={"name":indata}) 

## Specify the CMP code that is to be executed by the Timedata.RunTimeCode action 
Note how we use the inspect module to read the code from the _local_arima_fun_ function and remove all "#" comments from the text, since that will conflict with the SAS parser. In addition to the function definition, the generated Python code sets the PREDICT array by calling _local_arima_fun()_ with the input data set (Y). Besides running the code, objects are created to collect logs, variable status, and to store the code into a table. The stored code will be read in the next example of this tutorial.
    

In [ ]:
    rcode_file = os.path.join(CODE_PATH, "r_arima_code.r")

    cmpcode = """
      declare object py(PYTHON3) ;
      rc = py.Initialize() ;
      
      /* We'll be pushing the same code as the previous example, so PREDICT
         and Y will be used in the same way with the data given */
      rc = py.addVariable(revenue, 'ALIAS', 'Y') ;
      rc = py.addVariable(pyPred, 'ALIAS', 'PREDICT', 'READONLY', 'FALSE') ;
      rc = py.addVariable(_LENGTH_, 'ALIAS', 'NFOR') ;
      
      /* Push all code from Example 2. Note that this only pushes the code itself.
         The variables must still be specified. */
      declare object incode(INEXTCODE) ;
      rc = py.PushCodeFromTable(incode, "saspedia_extlang_ex2") ;

      rc = py.Run() ;

      declare object pylog(OUTEXTLOG) ;
      rc = pylog.Collect(py, 'EXECUTION') ;

      declare object pyvars(OUTEXTVARSTATUS) ;
      rc = pyvars.collect(py) ;

      /* Create R model */
      declare object robj(R) ;
      rc = robj.Initialize() ;
      
      
      rc = robj.PushCodeLine('library(astsa)') ;
      rc = robj.PushCodeLine('library(forecast)') ;
      rc = robj.PushCodeLine('') ;
      rc = robj.PushCodeLine('PREDICT <- seq(1.1, 156.1, by=1.0)') ;
      ** The Y passed in from SAS will contain the trailing missing values for the horizon, 
       which the R functions we use do not want there ;
      rc = robj.PushCodeLine("Y <- Y[1:(NFOR - HORIZON)]") ;
      rc = robj.PushCodeLine('Y_ts <- ts(Y, frequency=12)') ;
      **rc = robj.PushCodeLine('Y_ts <- window(Y_ts, 1, c(length(Y_ts), 12) )') ;
      rc = robj.PushCodeLine('log_Y_ts <- log(Y_ts)') ;
      rc = robj.PushCodeLine('') ;
      rc = robj.PushCodeLine('model <- stats::arima(log_Y_ts, order=c(p=0, d=1, q=1), seasonal=list(order=c(0,1,1), frequency=12))') ;

      **rc = robj.PushCodeLine('summary(model)') ;

      rc = robj.PushCodeLine('a <- stats::predict(model, n.ahead=HORIZON)') ;
      rc = robj.PushCodeLine('PREDICT <- c( exp(fitted.values(model)), exp(a$pred) )') ;
      
      rc = robj.addVariable(revenue, 'ALIAS', 'Y') ;
      rc = robj.addVariable(rPred, 'ALIAS', 'PREDICT', 'READONLY', 'FALSE') ;
      rc = robj.addVariable(_LENGTH_, 'ALIAS', 'NFOR') ;
      rc = robj.AddVariable(_LEAD_,'ALIAS','HORIZON') ;
      rc = robj.Run() ;

      declare object rlog(OUTEXTLOG) ;
      rc = rlog.Collect(robj, 'EXECUTION') ;
      declare object rvars(OUTEXTVARSTATUS) ;
      rc = rvars.collect(robj) ;
      
      /* Now the ATSM stuff */    
      
      /* Create external model specification for the Python model */
      declare object pyExmSpec(EXMSPEC);
      rc = pyExmSpec.open();
      rc = pyExmSpec.setOption('METHOD','PERFECT');
      rc = pyExmSpec.setOption('NLAGPCT',0);
      rc = pyExmSpec.setOption('NPARMS',2);
      rc = pyExmSpec.setOption('PREDICT','pyPred'); 
      rc = pyExmSpec.close();
      
      /* Create external model specification for the R model */
      declare object rExmSpec(EXMSPEC);
      rc = rExmSpec.open();
      rc = rExmSpec.setOption('METHOD','PERFECT');
      rc = rExmSpec.setOption('NLAGPCT',0);
      rc = rExmSpec.setOption('NPARMS',2);
      rc = rExmSpec.setOption('PREDICT','rPred'); 
      rc = rExmSpec.close();

      /* Specify data */
      declare object myData(tsdf);
      rc = myData.Initialize();
      rc = myData.AddY(revenue);
      rc = myData.SetOption('seasonality', 12);
      
      /* Add ESM and ARIMAX models to evaluate */
      declare object myDiagSpec(diagspec);
      rc = myDiagSpec.Open();
      rc = myDiagSpec.SetESM();
      rc = myDiagSpec.SetARIMAX();
      rc = myDiagSpec.Close();
      
      /* Auto-generate the best model for each model family */
      declare object myDiag(diagnose);
      rc = myDiag.Initialize(myData);
      rc = myDiag.SetSpec(myDiagSpec);
      rc = myDiag.Run();

      /* Add the external ARIMAX forecast to the dataframe */
      rc = myData.AddSeries(pyPred);
      rc = myData.AddSeries(rPred);

      /* Create a selection list and add external model */
      declare object mySel(SELSPEC);
      rc = mySel.open(2);
      rc = mySel.AddFrom(pyExmSpec);
      rc = mySel.AddFrom(rExmSpec);
      rc = mySel.SetOption('CRITERION', '{criterion}');
      rc = mySel.close();

      /* Run the forecast engine to select the best model */
      declare object myEng(foreng);
      rc = myEng.Initialize(myDiag);
      rc = myEng.AddFrom(mySel);
      rc = myEng.SetOption('HOLDOUT',12);
      rc = myEng.SetOption('CRITERION','{criterion}');
      rc = myEng.Run();

      /* Collect forecast output data */
      declare object modInfo(outmodelinfo);
      declare object fcst(outfor);
      declare object est(outest);
      declare object select(outselect);

      rc = modInfo.Collect(myEng);
      rc = fcst.Collect(myEng);
      rc = est.Collect(myEng);
      rc = select.Collect(myEng);
    """.format(rcode_file=rcode_file, criterion=MODEL_SELECTION_CRITERION)

In [ ]:
print(cmpcode)


## Specify the parameters of the action call 
Note that we promote the outcode table. This table will contain source code to be used in the next example. Therefore, you must use the same CAS session for that example. 

In [ ]:
    # runTimecode declaration gets hard to read with all the 
    # nested dicts, so use this shorthand for outer dicts and the
    # {} syntax for the innermost dicts
    d = dict
    # Shorthand for dicts that only have a key for "name"
    dname = lambda name: dict(name=name)

    # Define the timedata.runTimecode object
    res = conn.timedata.runtimecode(
                          table={
                              'name':'skinproduct',
                              'groupby':[#dname("ProductKey")], 
                                         dname("DistributionCenter")]
                                },
                          series=[d(accumulate='SUM', name='revenue')],
                          interval='Week',
                          #require=d(pkg="extlang"),
                          require=[d(pkg="atsm"), d(pkg="extlang"), d(pkg="tsm")],
                          timeid=d(name='date'),
                          lead=12,
                          arrayout={
                                    # specify output arrays
                                    'arrays':[dname("pyPred"), dname("rPred")],
                                    # specify table to put them in
                                    'table':d(name="outarray", replace=True)},
                          objin=[
                            d(table=d(name="outcode", groupby="DistributionCenter"), objRef="incode")
                                ],
                          objout=[
                            d(table=dname("outobj_pylog"), objRef="pylog"),
                            d(table=dname("outobj_pyvars"), objRef="pyvars"),
                            d(table=dname("outobj_rlog"), objRef="rlog"),
                            d(table=dname("outobj_rvars"), objRef="rvars"),
                            d(table=dname("outobj_modInfo"), objRef="modInfo"),
                            d(table=dname("outobj_fcst"), objRef="fcst"),
                            d(table=dname("outobj_select"), objRef="select"),
                            d(table=dname("outobj_est"), objRef="est")
                                 ],
                          code=cmpcode)
    del(d)
    print(res) # summary of action call


## Retrieve output tables

In [ ]:
    fcst_tbl = conn.CASTable("outobj_fcst")
    est_tbl = conn.CASTable("outobj_est")
    modinfo_tbl = conn.CASTable("outobj_modInfo")
    select_table = swat.CASTable("outobj_select")
    select_table.set_connection(conn)


## Print Python log
This should just print some output from the statsmodels package. 

In [ ]:
    outlog_tbl = conn.CASTable("outobj_pylog") 
    loglen = sum(outlog_tbl["_LOGLEN_"].values) 
    # Print log if it's not empty
    if loglen > 0: 
        text = "".join(outlog_tbl["_LOGTEXT_"].values) 
        print("LOG:") 
        print(text) 
        print() 

## Print R log

In [ ]:
    outlog_tbl = conn.CASTable("outobj_rlog") 
    loglen = sum(outlog_tbl["_LOGLEN_"].values) 
    # Print log if it's not empty
    if loglen > 0: 
        text = "".join(outlog_tbl["_LOGTEXT_"].values) 
        print("LOG:") 
        print(text) 
        print() 

## Check writable variables' status
If any of the writable variables were not updated (i.e. data was not brought back to the SAS program), print them out. 
This code should not produce any output.

In [ ]:
    varstats_tbl = conn.CASTable("outobj_pyvars") 
    varnames = varstats_tbl["_NAME_"].values 
    updated = varstats_tbl["UPDATED"].values 
    for varname,is_upd in zip(varnames,updated): 
        if int(is_upd) != 1: 
            print("WARNING :: Variable {0} was NOT updated".format(varname)) 

## Print model selection information

In [ ]:
    # Print model selection information
    num_entries_to_print = 100
    # NTS : Only one series, so no columns for the values
    column_names = ["_MODEL_", "_SELECTED_", "_LABEL_", MODEL_SELECTION_CRITERION]
    model_names, selected, descriptions, criterion_vals = \
        [select_table[col].values for col in column_names]
    #print("Summary of model selection. (Only showing first {0} entries. "\
    #      "Modify code to see all".format(num_entries_to_print))
    print("{0:-^75}".format(" MODEL SELECTION SUMMARY "))
    print("{0:^12}|{1:^6}|{2:^8}|{3:^50}"
          .format("Model", "Picked", MODEL_SELECTION_CRITERION, "Description"))
    print("-"*75)
    counter = collections.Counter()
    for i, (model, isSelected, desc, criterion) in \
        enumerate(zip( model_names, selected, descriptions, criterion_vals)):
        if isSelected == "YES":
            counter[model] += 1
        if i == num_entries_to_print:
            break
        print("{0:<12}|{1:<6}|{2:^8}|{3:50}"
              .format(model[:12], isSelected, round(criterion,3), desc))
    print("\n")


## Create bar chart depicting how many times each model won

In [ ]:
    objects = counter.keys()
    y_pos = np.arange(len(objects))
    plt.bar(y_pos, counter.values(), align='center')
    plt.xticks(y_pos, objects)
    plt.ylabel('Number of times it was Selected')
    plt.title('Model')
    plt.savefig("extlang_ex3_skin_selection.png")



## Tabulate a summary of the original and predicted values 


In [ ]:
    outarray_tbl = conn.CASTable("OUTARRAY")
    column_names = ["DATE", "DistributionCenter", "Revenue", "pypred", "rpred"]
    dates, distr_centers, orig_values, pypred_vals, rpred_vals = \
        [outarray_tbl[col].values for col in column_names]
    print('{0:-^72}'.format(' RESULTS '))
    print("{0:^12}|{1:^20}|{2:^20}|{3:^20}|{4:^20}".format(*column_names))
    print("-"*72)
    for i, (date, distr_center, revenue, pyPred, rPred) in \
        enumerate(zip( dates, distr_centers, orig_values, pypred_vals, rpred_vals)):
        print("{0:<12}|{1:^20}|{2:^20}|{3:^20}|{4:^20}"
              .format(str(date), distr_center, revenue, pyPred, rPred))
    print("\n")

## Plot the predicted and actual revenue for Miami, Chicago, and Atlanta 

In [ ]:
    from pandas.plotting import register_matplotlib_converters
    register_matplotlib_converters()
    ax = plt.gca()
    plt.ylabel("Revenue")
    for city,color in zip(["Philadelphia", "Las Vegas", "Detroit"],["green", "purple", "black"]): 
        idc = [i for i,ctr in enumerate(distr_centers) if ctr==city] 
        ctr_orig = orig_values[idc] 
        ctr_pypred = pypred_vals[idc] 
        ctr_rpred = rpred_vals[idc] 
        dates_1x = dates[idc]
        plt.scatter(dates_1x, ctr_orig, color=color, marker="o", lw=0.5,
                    label="Actual", facecolors="none", edgecolors=color)
        plt.plot(dates_1x, ctr_pypred, color="blue", ls="-", lw=0.5,
                 label="PythonForecast")
        plt.plot(dates_1x, ctr_rpred, color="red", ls="-", lw=0.5,
                 label="RForcast")
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, labels)
    # Set x ticks to be every 5 years so text doesn't get clobbered
    ax.xaxis.set_major_locator(mpldates.YearLocator(5))
    plt.savefig("extlang_ex3_skin.png")